In [ ]:
import os
import sys
import glob
# import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
# from sklearn.preprocessing import normalize
# from scipy.misc import imresize
# import seaborn as sns
from collections import defaultdict
import itertools

In [28]:
from os import listdir
from matplotlib import image
from PIL import Image

In [29]:
# preprocessing

In [ ]:
# directory_path = './Dataset/interim/real/'
# new_directory = './Dataset/preprocessed/real'
# # directory_path = './Dataset/interim/forged/'
# # new_directory = './Dataset/preprocessed/forged'
# for filename in listdir(directory_path):
#     # load image
#     image = Image.open(directory_path + '/' + filename)
#     # resize image and ignore original aspect ratio
#     img_resized = image.resize((200,200))
#     gs_image = img_resized.convert(mode='L')
#     # save
#     gs_image.save(new_directory + '/' + filename)

In [57]:
real_images = glob.glob('./Dataset/preprocessed/real/*.png')
forged_images = glob.glob('./Dataset/preprocessed/forged/*.png')

In [58]:
def get_image_id(image_path):
    """returns image ID from the image path"""
    image_id = image_path.split('/')[-1].split('_')[0]
    return image_id

In [59]:
# Create a dictionary to store all images.
real_images_dict = defaultdict(list)
forged_images_dict = defaultdict(list)

# Iterate over real images and put them in dictionary values for same image_id key.
for real_image, forged_image in zip(real_images, forged_images):
    
    # add image to dictionary
    real_image_id = get_image_id(real_image)
    real_images_dict[real_image_id].append(real_image)
    
    forged_image_id = get_image_id(forged_image)
    forged_images_dict[forged_image_id].append(forged_image)

In [60]:
# create tuples of image for training
negative_image_tuples = list()
# positive_image_tuples = list()

for image_id in real_images_dict.keys():
    real = real_images_dict[image_id]
    forged = forged_images_dict[image_id]
    
    negative_image_tuples.extend(list(itertools.product(real, real, forged)))
#     positive_image_tuples.extend(list(itertools.product(real, real)))

In [100]:
def process(image_path, size=(200, 220)):
    """returns processed images"""
    # Open image and convert to grayscale.
    image = Image.open(image_path)
    # image = image.convert("L")
    
    image_array = np.array(image)
    
    # Resize image to 128, 256 using bilinear interpolation.
    # image_array_processed = imresize(image_array, size=size, interp='bilinear')
    
    # Invert pixel values.
    image_array_processed = 1 - image_array
    
    # Normalize by dividing pixel values with standard deviation.
    image_array_processed = image_array_processed / np.std(image_array_processed)
    
    # Expand dimension to (200, 220, 1)
    image_array_processed = np.expand_dims(image_array_processed, axis=2)
    
    return image_array_processed

In [101]:
# pre-process data
image_1 = []
image_2 = []
image_3 = []
labels = []

for anchor, positive, negative in negative_image_tuples[:1000]:
    image_1.append(process(anchor))
    image_2.append(process(positive))
    image_3.append(process(negative))
    labels.append(0)
    
# for real1, real2 in positive_image_tuples:
#     image_1.append(process(real1))
#     image_2.append(process(real2))
#     labels.append(1)

In [102]:
# Convert to numpy arrays
image_1_array = np.asarray(image_1)
image_2_array = np.asarray(image_2)
image_3_array = np.asarray(image_3)
labels_array = np.array(labels)

In [103]:
# shuffle numpy arrays
idx = np.random.choice(range(len(image_1)), size=len(image_1), replace=False)

X_1 = image_1_array[idx]
X_2 = image_2_array[idx]
X_3 = image_3_array[idx]
y = labels_array[idx]

In [104]:
# split data into train-valid-test set.
train_split = 0.8
valid_split = 0.9
train_offset = int(train_split * len(X_1))
valid_offset = int(valid_split * len(X_1))

X_1_train = X_1[:train_offset]
X_2_train = X_2[:train_offset]
X_3_train = X_3[:train_offset]
y_train = y[:train_offset]

X_1_valid = X_1[train_offset:valid_offset]
X_2_valid = X_2[train_offset:valid_offset]
X_3_valid = X_3[train_offset:valid_offset]
y_valid = y[train_offset:valid_offset]

X_1_test = X_1[valid_offset:]
X_2_test = X_2[valid_offset:]
X_3_test = X_3[valid_offset:]
y_test = y[valid_offset:]

In [19]:
# y_train.shape, y_valid.shape, y_test.shape
X_1_train.shape, X_1_valid.shape, X_1_test.shape

((800, 200, 200, 1), (100, 200, 200, 1), (100, 200, 200, 1))

In [108]:
destn_dir = './Dataset/saved_processed/'
np.savez(os.path.join(destn_dir, 'train.npz'), X_1_train=X_1_train, X_2_train=X_2_train, X_3_train=X_3_train, y_train=y_train)
np.savez(os.path.join(destn_dir, 'valid.npz'), X_1_valid=X_1_valid, X_2_valid=X_2_valid, X_3_valid=X_3_valid, y_valid=y_valid)
np.savez(os.path.join(destn_dir, 'test.npz'), X_1_test=X_1_test, X_2_test=X_2_test, X_3_test=X_3_test, y_test=y_test)

In [117]:
y_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
# forgery prediction

In [11]:
import keras
from keras import layers, Input, Model, optimizers
import keras.backend as K
import numpy as np
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [22]:
data_path = './Dataset/saved_processed/'

train = np.load(os.path.join(data_path, 'train.npz'))
X_1_train=train['X_1_train']
X_2_train=train['X_2_train']
X_3_train=train['X_3_train']
y_train=train['y_train']

valid = np.load(os.path.join(data_path, 'valid.npz'))
X_1_valid=valid['X_1_valid']
X_2_valid=valid['X_2_valid']
X_3_valid=valid['X_3_valid']
y_valid=valid['y_valid']

test = np.load(os.path.join(data_path, 'test.npz'))
X_1_test=test['X_1_test']
X_2_test=test['X_2_test']
X_3_test=test['X_3_test']
y_test=test['y_test']

In [23]:
# try cnn

In [24]:
batch_size = 128
num_classes = 2
epochs = 2
input_shape = (200, 200, 1)

In [25]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [27]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(X_2_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_2_valid, y_valid))
score = model.evaluate(X_2_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 800 samples, validate on 100 samples
Epoch 1/2
800/800 [==============================] - 80s 100ms/step - loss: 0.1522 - accuracy: 0.9100 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/2
800/800 [==============================] - 77s 96ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Test loss: 0.0
Test accuracy: 1.0


In [134]:
# try siamese

In [135]:
from keras import layers, Input, Model, optimizers
import keras.backend as K
import numpy as np
import os

In [140]:
inputTensor = Input((200,200,1))

conv1 = layers.Conv2D(filters=96, 
                      kernel_size=(11,11), 
                      strides=1, 
                      activation='relu', 
                      input_shape=(155, 220, 1), 
                      data_format="channels_last")(inputTensor)

conv1_norm = layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001,center=True,
            scale=True, beta_initializer='zeros', gamma_initializer='ones',
            moving_mean_initializer='zeros',moving_variance_initializer='ones')(conv1)

conv1_pool = layers.MaxPooling2D(pool_size=(3,3), 
                                 strides=2)(conv1_norm)

conv2_padding = layers.ZeroPadding2D((2, 2))(conv1_pool)

conv2 = layers.Conv2D(filters=256, 
                      kernel_size=(5,5), 
                      strides=1, 
                      activation='relu')(conv2_padding)

conv2_norm = layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001,center=True,
            scale=True, beta_initializer='zeros', gamma_initializer='ones',
            moving_mean_initializer='zeros',moving_variance_initializer='ones')(conv2)

conv2_pool = layers.MaxPooling2D(pool_size=(3,3), 
                                 strides=2)(conv2_norm)

conv2_dropout = layers.Dropout(0.3, seed=1)(conv2_pool)

conv3_padding = layers.ZeroPadding2D((1,1))(conv2_dropout)

conv3 = layers.Conv2D(filters=384, 
                      kernel_size=(3,3), 
                      strides=1, 
                      activation = 'relu')(conv3_padding)

conv4_padding = layers.ZeroPadding2D((1,1))(conv3)

conv4 = layers.Conv2D(filters=256, 
                      kernel_size=(3,3), 
                      strides=1, 
                      activation='relu')(conv4_padding)
                                                                    
conv4_pool = layers.MaxPooling2D(pool_size=(3,3), 
                                 strides=2)(conv4)
                                                                    
conv4_dropout = layers.Dropout(0.3, seed=1)(conv4_pool)
                                                                    
flatten_layer = layers.Flatten()(conv4_dropout)
                                                                    
fully_connected1 = layers.Dense(1024)(flatten_layer)

fc1_dropout = layers.Dropout(0.5, seed=1)(fully_connected1)
                                                                    
embedding = layers.Dense(128)(fc1_dropout)
                                                                    
embedding_model = Model(inputs=[inputTensor], 
                         outputs=embedding, 
                         name='embedding_model')
                                                                    
embedding_model.summary()

Model: "embedding_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 200, 200, 1)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 190, 190, 96)      11712     
_________________________________________________________________
batch_normalization_3 (Batch (None, 190, 190, 96)      384       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 94, 94, 96)        0         
_________________________________________________________________
zero_padding2d_4 (ZeroPaddin (None, 98, 98, 96)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 94, 94, 256)       614656    
_________________________________________________________________
batch_normalization_4 (Batch (None, 94, 94, 256)   

In [137]:
def euclidean_distance_loss(y_true, y_pred):
    """
    :param y_true: TensorFlow/Theano tensor
    :param y_pred: TensorFlow/Theano tensor of the same shape as y_true
    :return: float
    """
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))

In [138]:
def triplet_loss(embeddings):
    """
    calculates triplet loss over inputs.
    """
    
    processed_a, processed_p, processed_n = embeddings[0], embeddings[1], embeddings[2]
    
    positive_dist= euclidean_distance_loss(processed_a, processed_p)
    negative_dist = euclidean_distance_loss(processed_a, processed_n)
       
    margin = 0.0
    loss = K.maximum(margin, positive_dist - negative_dist)
    
    return K.mean(loss)

In [139]:
def identity_loss(y_true, y_pred):
    """
    Fake loss function for Keras.
    """
    return y_pred - 0 * y_true

In [141]:
# Siamese model

in_dim=(200,200,1)
input_anchor = Input(shape=(in_dim))
input_positive = Input(shape=(in_dim))
input_negative = Input(shape=(in_dim))
embedding_a=embedding_model(input_anchor)
embedding_p=embedding_model(input_positive)
embedding_n=embedding_model(input_negative)


# https://github.com/maciejkula/triplet_recommendations_keras/blob/master/triplet_keras.ipynb
# https://www.kaggle.com/kmader/image-similarity-with-siamese-networks
# https://github.com/keras-team/keras/issues/9498
# https://github.com/keras-team/keras/issues/3921#issuecomment-250643688
# 

# NOTE: layers.merge is deprecated. 
# the only way to do it now is custom keras layer which implements the triplet loss.
# Create a loss layer
# loss = layers.merge([embedding_a, embedding_p, embedding_n], 
#                     mode=triplet_loss, 
#                     name='loss',)


# Write a custom layer for loss function.
# loss = layers.triplet_loss([embedding_a, embedding_p, embedding_n])

embedding_concat = layers.concatenate(inputs=[embedding_a, 
                                    embedding_p, 
                                    embedding_n], axis=-1)

loss_layer = layers.Lambda(function=triplet_loss, 
                     output_shape=(1,))

loss = loss_layer(embedding_concat)

siamese_model = Model(input=[input_anchor, input_positive, input_negative], 
                      output=loss)

siamese_model.compile(loss=identity_loss, optimizer=optimizers.Adam())

/Users/huamingsun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("la...)`


In [142]:
siamese_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 200, 200, 1)  0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 200, 200, 1)  0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 200, 200, 1)  0                                            
__________________________________________________________________________________________________
embedding_model (Model)         (None, 128)          129407808   input_3[0][0]                    
                                                                 input_4[0][0]              

In [143]:
siamese_model.fit(x=[X_1_train, X_2_train, X_3_train], y=y_train, batch_size=16,)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/1
800/800 [==============================] - 2078s 3s/step - loss: 1683.7408


In [145]:
siamese_model.evaluate(x=[X_1_test, X_2_test, X_3_test], y=y_test)

100/100 [==============================] - 35s 354ms/step
